In [1]:
# Digit recognizer inspired by LeNet5

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf 

import keras 
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation, Dropout
from keras.optimizers import Adam

from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


from sklearn.model_selection import train_test_split

/anaconda3/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda3/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.DeprecatedIn23,
Using TensorFlow backend.


In [3]:
np.random.seed(2)

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train=np.array(train.drop(['label'], axis=1))
Y_train=np.array(train['label'])

In [7]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_test=np.array(test)

In [9]:
X_train.shape

(42000, 784)

In [10]:
X_test.shape

(28000, 784)

In [11]:
#Rescale data
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255
X_train[0].shape

(784,)

In [12]:
#Reshape datasets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [13]:
#Add padding to get 32x32 image
X_train = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

In [14]:
#One-hot encoding the labels
Y_train = to_categorical(Y_train, num_classes=10)

In [15]:
model = Sequential()

#Conv layer 1 
#input 32x32x1, output 28x28x6
model.add(Conv2D(32,(5,5), padding = 'Same', activation = 'relu', input_shape = (32,32,1)))
model.add(BatchNormalization())

#Pooling layer 1
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))



#Conv layer 2
model.add(Conv2D(64, (5,5), padding = 'Same', activation = 'relu'))
model.add(BatchNormalization())


#Pooling layer 2
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#Conv layer 3
model.add(Conv2D(128, (3,3), padding = 'Same', activation = 'relu'))
model.add(BatchNormalization())

#Pooling layer 3
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))



#Flatten
model.add(Flatten())



#Fully connected layer 1
model.add(Dense(128, activation = 'relu'))


#Fully connected layer 2
model.add(Dense(64, activation = 'relu'))


#Output Layer
model.add(Dense(10))
model.add(Activation(tf.nn.softmax))


model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [16]:
#Split the train dataset to training and  validation 
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [17]:
model.fit(X_train, Y_train, batch_size=100, epochs=30, validation_data=(X_val, Y_val))


Train on 37800 samples, validate on 4200 samples
Epoch 1/30
37800/37800 [==============================] - 183s 5ms/step - loss: 0.3300 - acc: 0.8951 - val_loss: 0.0892 - val_acc: 0.9714
Epoch 2/30
37800/37800 [==============================] - 195s 5ms/step - loss: 0.0836 - acc: 0.9748 - val_loss: 0.0632 - val_acc: 0.9819
Epoch 3/30
37800/37800 [==============================] - 232s 6ms/step - loss: 0.0606 - acc: 0.9807 - val_loss: 0.0676 - val_acc: 0.9810
Epoch 4/30
37800/37800 [==============================] - 211s 6ms/step - loss: 0.0509 - acc: 0.9842 - val_loss: 0.0421 - val_acc: 0.9883
Epoch 5/30
37800/37800 [==============================] - 169s 4ms/step - loss: 0.0475 - acc: 0.9849 - val_loss: 0.0375 - val_acc: 0.9900
Epoch 6/30
37800/37800 [==============================] - 166s 4ms/step - loss: 0.0409 - acc: 0.9869 - val_loss: 0.0366 - val_acc: 0.9898
Epoch 7/30
37800/37800 [==============================] - 162s 4ms/step - loss: 0.0360 - acc: 0.9881 - val_loss: 0.0434 - v

In [18]:
Y_pred = model.predict_classes(X_test)

In [19]:
pd.DataFrame({"ImageId": list(range(1,len(Y_pred)+1)), "Label": Y_pred}).to_csv('submission.csv', index=False, header=True)